In [1]:
# Import libraries
import glob
import pandas as pd

In [2]:
path = "../data/influencer_tweets/"

csv_files = glob.glob(path + "/*.csv")

# Read each CSV file into DataFrame
# This creates a list of dataframes
df_list = (pd.read_csv(file) for file in csv_files)

# Concatenate all DataFrames
df   = pd.concat(df_list, ignore_index=True)

In [3]:
df.shape

(56100, 6)

In [4]:
df.head()

,id,userID,created_at,full_text,favorite_count,retweet_count
0,1644820236749635591,BRKsEDU,2023-04-08 21:51:18+00:00,@MarioThael Muito obrigado!!! Quem editou foi ...,4,1
1,1644800548812587008,BRKsEDU,2023-04-08 20:33:04+00:00,@V_JuniorR Valeu,1,0
2,1644800516403126274,BRKsEDU,2023-04-08 20:32:56+00:00,@vinossauro Sim e sim 😀,3,0
3,1644800453090091011,BRKsEDU,2023-04-08 20:32:41+00:00,@vsouzaduart Muito obrigado!!!,1,0
4,1644789241619120133,BRKsEDU,2023-04-08 19:48:08+00:00,Acabei de publicar no canal minha crítica ao f...,751,21
